# C5. Comprima todos los textos de las reviews obteniendo un ratio de compresión mejor que 2.

    ¿De cuanto es el ratio de compresión?
    ¿Cuánto tarda en comprimir y descomprimir (por separado)? (use el magic %%timeit)
    ¿Cuánto ocupa cada carácter en promedio una vez comprimido?
    Si tomamos la entropía base dos para los caracteres ¿cuánto da? ¿cuantos bytes por caracter son esos?
    Si utilizaramos un compresor aritmético por caracter, aproximadamente ¿cuál sería el ratio de compresión en el caso más optimista?
    ¿Cúal algoritmo de compresión de los dos sería mejor?



In [25]:
import pandas as pd
import numpy as np

reviews = pd.read_csv('Dataset/review.csv', usecols=['text'])

In [26]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3838105 entries, 0 to 3838104
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
dtypes: object(1)
memory usage: 29.3+ MB


In [27]:
df = reviews
df

,text
0,"If you decide to eat here, just be aware it is..."
1,Cute interior and owner (?) gave us tour of up...
2,I was really between 3 and 4 stars for this on...
3,First time there and it was excellent!!! It fe...
4,"Great burgers,fries and salad! Burgers have a..."
...,...
3838100,We redesigned my moms dress and mad it complet...
3838101,"This spot offers a great, affordable east week..."
3838102,This Home Depot won me over when I needed to g...
3838103,For when I'm feeling like ignoring my calorie-...


In [28]:
reviews_text = df['text'].str.cat(sep='. ', na_rep=' ')

In [29]:
len(reviews_text)

2022941902

In [30]:
import zstandard as zstd
cctx = zstd.ZstdCompressor()
compressed_text = cctx.compress(reviews_text.encode())

In [31]:
dctx = zstd.ZstdDecompressor()
decompressed_text = dctx.decompress(compressed_text)

In [32]:
# df.memory_usage(deep=True)
from sys import getsizeof
input_memory_usage = getsizeof(reviews_text)
output_memory_usage = getsizeof(compressed_text)
input_memory_usage/output_memory_usage

5.468967505794416

# 5. Comprima todos los textos de las reviews obteniendo un ratio de compresión mejor que 2.

## ¿De cuanto es el ratio de compresión?
    
   

In [33]:
print(input_memory_usage/output_memory_usage)

5.468967505794416



    
## ¿Cuánto tarda en comprimir y descomprimir (por separado)? (use el magic %%timeit)
    
   

In [34]:
%timeit cctx.compress(reviews_text.encode())

26.7 s ± 357 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit dctx.decompress(compressed_text)

4.78 s ± 46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


 
## ¿Cuánto ocupa cada carácter en promedio una vez comprimido?
  

In [36]:
print((input_memory_usage/output_memory_usage) / len(reviews_text))

2.7034723539946805e-09


## Si tomamos la entropía base dos para los caracteres ¿cuánto da? ¿cuantos bytes por caracter son esos?

Suponiendo que tengo 256 caracteres ascii y que todos son equiprobables (un supuesto muy fuerte). Podemos calcular:

![Entropia Shannon](entropia-shannon.png)

donde i son los 256 caracteres ascii y P_i es 1/256 porque lo asumo equiprobable.

In [39]:
import math

h = -( (1/256) * math.log2(1/256) * 256 )
print(h)

8.0


8 es la cantidad de bits por caracter. Eso significa que es 1 byte por caracter.

  
## Si utilizaramos un compresor aritmético por caracter, aproximadamente ¿cuál sería el ratio de compresión en el caso más optimista?  
    
Los compresores aritméticos en el caso más optimista se acercan a la longitud ideal que es -log_2 (P_i)

In [41]:
- math.log2(1/256)

8.0

## ¿Cúal algoritmo de compresión de los dos sería mejor?

Bueno en estos cálculos dio lo mismo por las asunciones que hicimos. Pero conociendo la naturaleza del texto que son reviews donde las palabras tienden a repetirse y ser parecidas entre una review y otra, la compresión aritmética debería dar mejor.